In [ ]:
import picos as pic
import cvxopt as cvx

import numpy as np
from numpy import linalg as LA
from scipy.special import comb as nchoosek # n Choose k (n ueber k)

In [ ]:
### Seite 37 (im Text zwischen (4.12) und (4.13))
def _N(n):
    return np.diag([p for p in xrange(-n, 0, 1)])

### Seite 55; (4.64)
def _M(n):
    return np.diag([p for p in xrange(0, n, 1)])

### Seite 55; (4.64)
def _P(l, k, n):
    I = np.eye(n)
    Mn = _M(n)
    P = I
    if k == 0:
        pass # P=I
    else:
        for q in xrange(0, k, 1):
            P = P * ((l-q)*I + Mn)
    return cvx.matrix(P)

In [ ]:
##############################
# Boris Paper UBoot          #
##############################
A = np.matrix([[0, 1, 0],
              [0, 0, 1],
              [0, 0, -0.005]])
a = -A[-1][:].T ### ?????????????????????
b = np.matrix([[0],[0],[1]])
c = np.matrix([1, 0, 0])
u_max = 2.5e-5
n = 3

X0 = [np.matrix([-10, -0.05, -0.0046]).T,
      np.matrix([-10, -0.05, 0.0046]).T,
      np.matrix([-10, 0.05, -0.0046]).T,
      np.matrix([10, -0.05, 0.0046]).T,
      np.matrix([10, -0.05, -0.0046]).T,
      np.matrix([10, 0.05, 0.0046]).T]

#print "A:\n", A
#print "a:\n", a
#print "b:\n", b
#print "c:\n", c

N = cvx.matrix(_N(n))
M = cvx.matrix(_M(n))

##### Entwurf parameter #####
beta = 1 # beta >=1 !

##############################
# Convex Problem (35)        #
##############################
prob = pic.Problem()

# Constants
AA = pic.new_param('A', A)
II = pic.new_param('I_n', np.eye(n+1))
III = pic.new_param('I_n+1', np.eye(n+1))
aa = pic.new_param('a', a)
bb = pic.new_param('b', b)
XX0 = pic.new_param('X0', X0)

NN = pic.new_param('N', N)
MM = pic.new_param('M', M)

AA0 = pic.new_param('A0', AA+bb*aa.T)

# Problem
prob = pic.Problem()

# Parameters
QQ = prob.add_variable('Q', (n, n), vtype='symmetric')
zz = prob.add_variable('z', n)
zz_star = prob.add_variable('z_star', n)
delta = prob.add_variable('delta', 1)

# Objective
prob.set_objective('max', delta)

# Constraints
prob.add_constraint(QQ >> 0)
prob.add_constraint(QQ*NN + NN*QQ << 0)
prob.add_constraint(QQ*AA0.T + AA0*QQ - zz*bb.T - bb*zz.T << 0)

## (31) ???

prob.add_list_of_constraints([((1          & XX0[i].T) //
                               (XX0[i]     & QQ      )) >> 0
                                   for i in range(0, len(X0))])

prob.add_constraint(QQ*AA0.T + AA0*QQ - zz_star*bb.T - bb*zz_star.T << -2*delta*QQ)

prob.add_constraint(((beta**2*u_max**2 & zz_star.T-aa.T*QQ) //
                     (zz_star-QQ*aa    & QQ)) >> 0)

print prob

#prob.solve(verbose=0, solver='cvxopt')

#print "P:\n", PP
#print "eig:\n", LA.eigvals(PP.value)

#print 
#print "AP+PA:\n",(AA.T*PP + PP*AA).value
#print "eig:\n", LA.eigvals((AA.T*PP + PP*AA).value)